This is part of the supporting information for the paper  
*ParAMS: Parameter Optimization for Atomistic and Molecular Models* (DOI: *123123*)  
The full documentation can be found at https://www.scm.com/doc.trunk/params/index.html

# Optimization: Original Setup

This Notebook sets up the optimization of the [Mue2016](https://doi.org/10.1021/acs.jctc.6b00461) force field for ReaxFF as published by Müller and Hartke (MH). It aims to retain most of the settings that were discussed in the original publication for the sake of a comparison. Specifically, the same parameters will be optimizied within the same bounds as in the original publication. This setup differs from the MH publication in the following:

* The initial point $x_0$ for this optimization is the already optimized force field as found by MH, possibly giving this optimization an advantage
* Optimizer related settings could not be considered as we are using a differend algorithm altogether: CMA-ES rather than OGOLEM

This notebook assumes that you have MH's supporting information downloaded and extracted into *../MH*
 

In [7]:
import os, sys
import numpy as np
from os.path    import join as opj
from scm.params import *
from scm.params.experimental import ActiveParameterSearch
from scm.params import __version__ as paramsver
print(f"ParAMS Version used: {paramsver}")

INDIR = '../data/reaxff'
if not os.path.exists(INDIR):
    os.makedirs(INDIR)

ParAMS Version used: 0.5.0



# Step 0: Auxiliary functions
Müller and Hartke provide the reference gradients as external files. This function adds them to the data set:

In [8]:
def add_grads(path, dataset):
    for i in os.listdir(path):
        if i.endswith('gradient'):
            name = i.rstrip('.gradient')
            grads = np.loadtxt(opj(path, i), skiprows=1, usecols=(1,2,3))
            for id,at in enumerate(grads):
                for xyz,value in enumerate(at):
                    dataset.add_entry(f'forces("{name}", {id}, {xyz})', 1., sigma=0.01, reference=-value)

Evaluate and print force field losses for the training and validation sets:

In [9]:
def printloss(ffpath):
    params  = ReaxParams(ffpath)
    results = jc.run(params)
    vsloss  = val_set.evaluate(results, loss='sse')
    tsloss  = train_set.evaluate(results, loss='sse')
    print(f"Training   Set loss: {tsloss:.2f}")
    print(f"Validation Set loss: {vsloss:.2f}")

Load the Mue2016 force field with proper ranges:

In [19]:
def get_mue16():
    ff     = ReaxParams('../MH/mue2016')
    ffbool = ReaxParams('../MH/ffield_bool')
    ffmin  = ReaxParams('../MH/ffield_min')
    ffmax  = ReaxParams('../MH/ffield_max')
    ff.range = [(pmi.value, pma.value) for pmi,pma in zip(ffmin, ffmax)]
    ff.is_active = [True if p == 1 else False for p in ffbool.x]
    return ff


# Step 1: Convert from the old ReaxFF format to ParAMS

We start with the job collection:

In [11]:
jc1 = geo_to_params('../MH/optInput/geo', normal_run_settings='../MH/control')
jc2 = geo_to_params('../MH/valSet/geo',   normal_run_settings='../MH/control')

print('The following jobIDs are in *both*, the training and validation set:\n')
print("\n".join([i for i in jc1.keys() if i in jc2])+'\n')

The following jobIDs are in *both*, the training and validation set:

dmds
s8
dpds
dpods



Join the sets into one job collection, tell AMS that Gradients need to be computed and append the link to the original publication in the metadata:

In [12]:
jc  = jc1 + jc2

for e in jc.values():
    e.metadata['Source'] = 'https://doi.org/10.1021/acs.jctc.6b00461'
    e.settings.input.ams.properties.gradients = True 
    
jc.store(opj(INDIR, 'jobcollection.yml'))

Now convert the data sets:

In [13]:
train_set = trainset_to_params('../MH/optInput/trainset.in')
val_set  =  trainset_to_params('../MH/valSet/trainset.in')
add_grads('../MH/optInput/grads', train_set)

for ds in [train_set, val_set]:
    for e in ds:
        e.metadata['Source'] = 'https://doi.org/10.1021/acs.jctc.6b00461'

train_set.store(opj(INDIR, 'trainingset.yml'))
val_set.store(  opj(INDIR, 'validationset.yml'))

# Step 2: Calculate the loss value of Mue2016

In [8]:
printloss('../MH/mue2016')
print()
print('Published training set value is 12393\n(https://doi.org/10.1021/acs.jctc.6b00461)\n')
print('A more recent publication reports a training set value of 16271\n(https://doi.org/10.1021/acs.jctc.9b00769)')

Training   Set loss: 14441.00
Validation Set loss: 14450.95

Published training set value is 12393
(https://doi.org/10.1021/acs.jctc.6b00461)

A more recent publication reports a training set value of 16271
(https://doi.org/10.1021/acs.jctc.9b00769)


# Step 3: Select the most sensitive parameters
The parameter interface we want to optimize:

In [14]:
x0 = ReaxParams('../MH/mue2016', bounds_scale=1.2)

The following translates to a one-dimensional scan of every parameter to determine which ones produce the highest response in the validation set's loss value.  It is also possible to scan all possible $k$ parameter combinations, although this would drastically increase the computational time:

In [15]:
if not os.path.exists(opj(INDIR, 'aps.npz')):
    print('Searching for the most sensitive parameters ...')
    aps    = ActiveParameterSearch(x0, val_set, jc)
    aps.scan(steps=[1.05], verbose=False)
    aps.save(opj(INDIR, 'aps.npz'))
else:
    print('Loading previous results')
    aps = ActiveParameterSearch(x0, val_set, jc, opj(INDIR, 'aps.npz'))
print('Done')

Loading previous results
Done


Select the most sensitive 35 parameters for optimization:

In [16]:
print(f"Total number of paramters in force field: {len(x0)}")
print(f"Number of paramters to be optimized before setting: {len(x0.active)}")
x0.is_active = aps.get_is_active(35)
print(f"Number of paramters to be optimized after setting:  {len(x0.active)}")

Total number of paramters in force field: 701
Number of paramters to be optimized before setting: 619
Number of paramters to be optimized after setting:  35


Print the selected paramter names (for more details regarding the paramters, refer to the [ReaxFF Documentation](https://www.scm.com/doc/ReaxFF/ffield_descrp.html#ffield)):

In [17]:
for name in x0.active.names:
    print(name)

C:Val_i;;3a,4b,5,9a;;Valency
C:r_0^pi;;2;;Pi bond covalent radius
C:alpha_ij;;23b;;van der Waals parameter
C:p_val3;;13b,13a;;Valence angle parameter
H:r_vdW;;23a;;van der Waals radius
H:Val_i^e;;7,8,9;;Number of valence electrons
H:chi_i;;24,25;;EEM electronegativity
S:r_0^sigma;;2;;Sigma bond covalent radius
S:r_vdW;;23a;;van der Waals radius
S:alpha_ij;;23b;;van der Waals parameter
S:chi_i;;24,25;;EEM electronegativity
C.S:p_bo2;;2;;Sigma bond order
S.S:p_bo2;;2;;Sigma bond order
C.H:r_vdW;;23a;;VdW radius
C.H:alpha_ij;;23a;;VdW parameter
C.H:r_0^sigma;;2;;Sigma bond length
C.O:alpha_ij;;23a;;VdW parameter
C.O:r_0^pi;;2;;Pi bond length
C.S:r_vdW;;23a;;VdW radius
C.S:alpha_ij;;23a;;VdW parameter
C.S:r_0^sigma;;2;;Sigma bond length
H.S:r_vdW;;23a;;VdW radius
H.S:alpha_ij;;23a;;VdW parameter
H.S:r_0^sigma;;2;;Sigma bond length
C.C.S:Theta_0,0;;13g;;180o-(equilibrium angle)
C.C.S:p_coa1;;15;;Valence conjugation
C.O.H:p_val1;;13a;;Valence angle parameter
C.S.H:p_val7;;13c;;Undercoordinat

Print the intersection between the parameters that will be optimized and the parameters that MH optimized:

In [20]:
mue16 = get_mue16()
for name in set(mue16.active.names) & set(x0.active.names):
    print(name)

H.S:r_vdW;;23a;;VdW radius
C.S:r_vdW;;23a;;VdW radius
C:p_val3;;13b,13a;;Valence angle parameter
H.S:alpha_ij;;23a;;VdW parameter
S.S:p_bo2;;2;;Sigma bond order
S:r_0^sigma;;2;;Sigma bond covalent radius
C.S.S:Theta_0,0;;13g;;180o-(equilibrium angle)
H.S:r_0^sigma;;2;;Sigma bond length
C.S:p_bo2;;2;;Sigma bond order
C.C.S:Theta_0,0;;13g;;180o-(equilibrium angle)
C.S:r_0^sigma;;2;;Sigma bond length
S:r_vdW;;23a;;van der Waals radius
C.S:alpha_ij;;23a;;VdW parameter
C.S.S.C:V_2;;16a;;V2-torsion barrier
S:alpha_ij;;23b;;van der Waals parameter
C.S.S.C:p_tor1;;16a;;Torsion angle parameter


# Step 4: Start the optimization
Note that in the following cell, the timeout is set to 60 seconds for demonstrational purposes.

In [13]:
o            = CMAOptimizer(popsize=36, sigma=0.3)
callbacks    = [Logger(), Timeout(60), TimePerEval(10), EarlyStopping(6000, watch='validationset')]
optimization = Optimization(jc, [train_set, val_set], x0, o, callbacks=callbacks)

In [14]:
optimization.summary()

Optimization() Instance Settings:
Workdir:                           /home/opt
JobCollection size:                458
Interface:                         ReaxParams
Active parameters:                 35
Optimizer:                         CMAOptimizer
Parallelism:                       ParallelLevels(optimizations=1, parametervectors=6, jobs=1, processes=1, threads=1)
Verbose:                           True
Callbacks:                         Logger
                                   Timeout
                                   TimePerEval
                                   EarlyStopping
PLAMS workdir path:                /tmp

Evaluators:
-----------
Name:                              trainingset (_LossEvaluator)
Loss:                              SSE
Evaluation frequency:              1

Data Set entries:                  4875
Data Set jobs:                     231
Batch size:                        None

Use PIPE:                          True
---
Name:                              valid

Start the optimization:

In [15]:
result = optimization.optimize()

[2021-02-13 17:19:17] Starting parameter optimization.
[2021-02-13 17:19:21] Initial loss: 1.444e+04
[2021-02-13 17:19:27] Best trainingset loss: 7.261e+06
[2021-02-13 17:19:28] Best trainingset loss: 2.535e+06
[2021-02-13 17:19:30] Best trainingset loss: 3.535e+05
[2021-02-13 17:19:57] Time per f-evaluation (trainingset): 0:00:03.749031
[2021-02-13 17:20:13] Best trainingset loss: 2.006e+05
[2021-02-13 17:20:14] Time per f-evaluation (validationset): 0:00:03.373102
[2021-02-13 17:20:22] Best trainingset loss: 1.570e+05
[2021-02-13 17:20:29] Callback: Timeout
[2021-02-13 17:20:32] Time per f-evaluation (trainingset): 0:00:03.213117
[2021-02-13 17:20:42] Time per f-evaluation (validationset): 0:00:03.420483
[2021-02-13 17:20:49] Optimization done after 0:01:32
[2021-02-13 17:20:49] Final loss: 1.570e+05


# Step 5: Calculate the loss value of MueParAMS

In [16]:
printloss(opj(INDIR,'MueParAMS.ff'))

Training   Set loss: 11876.57
Validation Set loss: 5376.82
